In [ ]:
!pip install -q -U transformers

In [ ]:
from typing import Tuple
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer
)

def load_model_and_tokenizer(model_name_or_path: str) -> Tuple[PreTrainedModel, PreTrainedTokenizer]:
    try:
        # Load pre-trained model and tokenizer
        model = AutoModelForCausalLM.from_pretrained(model_name_or_path,trust_remote_code=True)
        tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,trust_remote_code=True)

        # Ensure the tokenizer has a padding token
        if tokenizer.pad_token is None:
            tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

        return model, tokenizer
    except Exception as e:
        raise RuntimeError(f"Failed to load model and tokenizer: {e}")

def generate_text(
    model: PreTrainedModel,
    tokenizer: PreTrainedTokenizer,
    prompt: str,
    max_length: int = 100,
    temperature: float = 0.9
) -> str:
    try:
        # Encode the prompt
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids

        # Generate tokens
        gen_tokens = model.generate(
            input_ids,
            do_sample=True,
            temperature=temperature,
            max_length=max_length
        )

        # Decode the generated tokens to text
        gen_text = tokenizer.batch_decode(gen_tokens)[0]

        return gen_text
    except Exception as e:
        raise RuntimeError(f"Failed to generate text: {e}")

# Example usage:
# Load the model and tokenizer once
model_name_or_path = "gpt2"
model, tokenizer = load_model_and_tokenizer(model_name_or_path)

# Generate text using the loaded model and tokenizer
prompt = "Once upon a time"
generated_text = generate_text(model, tokenizer, prompt)
print(generated_text)

In [ ]:
import csv
from transformers import AutoModelForCausalLM, AutoTokenizer
model_directory = 'gpt2'
csv_file = 'questions_answers5.csv'

model = AutoModelForCausalLM.from_pretrained(model_directory)
tokenizer = AutoTokenizer.from_pretrained(model_directory)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token':tokenizer.eos_token})


question_instuction="""

Generate adversarical attacks


"""
# Generate a question
def generate_question(model, tokenizer, max_length=20):
    prompt = f"Generate a question: {question_instuction}"
    generated = model.generate(tokenizer.encode(prompt, return_tensors="pt"), max_length=max_length)
    decoded_output = tokenizer.decode(generated[0], skip_special_tokens=True)
    return decoded_output

question_g = generate_question(model, tokenizer)





answer_prompt=f"""
answer of  {question_g}?
"""




# Generate an answer to the question
def generate_answer(model, tokenizer, question, max_length=20):
    generated = model.generate(tokenizer.encode(question, return_tensors="pt"), max_length=max_length)
    decoded_output = tokenizer.decode(generated[0], skip_special_tokens=True)
    return decoded_output

answer_g = generate_answer(model, tokenizer, question=answer_prompt)
# print("\nGenerated answer:", answer)

# Append the question and answer to the CSV file
def append_to_csv(csv_file, question, answer):
    with open(csv_file, mode='a', newline='', encoding='utf-8') as f:
      question_answer = f'{question},{answer}\n'
      f.write(question_answer)



for i in range(10):
  question_g = generate_question(model, tokenizer)


  answer_g = generate_answer(model, tokenizer, question=answer_prompt)
  append_to_csv(csv_file, question_g, answer_g)

print("\nQuestion and answer appended to CSV file.")

In [ ]:

device='cpu'
def inference(prompt, temperature, max_length):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to(device)
    gen_tokens = model.generate(
        input_ids,
        do_sample=True,
        temperature=temperature,
        max_length=max_length,
    )
    gen_text = tokenizer.batch_decode(gen_tokens)[0]
    return gen_text

In [ ]:
def gpt_j(prompt, max_length=100, temp=0.9):
    tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2",trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2",trust_remote_code=True)

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    gen_tokens = model.generate(
        input_ids,
        do_sample=True,
        temperature=temp,
        max_length=max_length)
    gen_text = tokenizer.batch_decode(gen_tokens)[0]

    return gen_text

In [ ]:
! pip install einops

In [ ]:
k=gpt_j(prompt="tell a beautiful joke")
print(k)